In [2]:
import os
from collections import Counter

txt_dir = r'C:\Users\91955\Downloads\Complete-Blood-Cell-Count-Dataset-master\train\labels'

label_counter = Counter()

for txt_filename in os.listdir(txt_dir):
    if txt_filename.endswith('.txt'):
        with open(os.path.join(txt_dir, txt_filename), 'r') as file:
            for line in file:
                label = line.split()[0]  # Extract the label (first column)
                label_counter[label] += 1  # Count the occurrence

for label, count in label_counter.items():
    print(f"Label: {label}, Count: {count}")

print(f"Total unique labels: {len(label_counter)}")


Label: 1, Count: 307
Label: 0, Count: 3341
Label: 2, Count: 304
Total unique labels: 3


In [ ]:
import os
import xml.etree.ElementTree as ET

# Define the paths
input_dir = r'C:\Users\91955\Downloads\Complete-Blood-Cell-Count-Dataset-master\valid\label'
images_dir = r'C:\Users\91955\Downloads\Complete-Blood-Cell-Count-Dataset-master\valid\images'
output_dir = r'C:\Users\91955\Downloads\Complete-Blood-Cell-Count-Dataset-master\\valid\label'

class_names = ['RBC', 'WBC', 'Platelets']

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def convert_bbox_format(xmin, ymin, xmax, ymax, img_width, img_height):
    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return x_center, y_center, width, height

image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for filename in os.listdir(input_dir):
    if filename.endswith('.xml'):
        xml_file = os.path.join(input_dir, filename)
        tree = ET.parse(xml_file)
        root = tree.getroot()

        image_filename = root.find('filename').text
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            label_file = os.path.join(output_dir, filename.replace('.xml', '.txt'))
            with open(label_file, 'w') as f:
                pass  
            print(f"Placeholder label file created for missing image: {image_filename}")
            continue

        img_width = int(root.find('size/width').text)
        img_height = int(root.find('size/height').text)

        yolo_annotations = []

        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name in class_names:
                class_id = class_names.index(class_name)
                xmin = int(obj.find('bndbox/xmin').text)
                ymin = int(obj.find('bndbox/ymin').text)
                xmax = int(obj.find('bndbox/xmax').text)
                ymax = int(obj.find('bndbox/ymax').text)
                
                x_center, y_center, width, height = convert_bbox_format(xmin, ymin, xmax, ymax, img_width, img_height)
                
                yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

        if yolo_annotations:
            yolo_file = os.path.join(output_dir, filename.replace('.xml', '.txt'))
            with open(yolo_file, 'w') as f:
                f.write("\n".join(yolo_annotations))

print("Conversion completed!")
